In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

In [2]:
data = pd.read_csv('../data_merged/merged_total.csv') # +- 1 minuut om in te laden

C:\Users\buyse\AppData\Local\Temp\ipykernel_14096\1816359094.py:1: DtypeWarning: Columns (24,25,85,106) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data_merged/merged_total.csv')


### Onnodige kolommen weglaten

In [5]:
data = data[data['account_adres_geografische_regio'] == 'Oost-Vlaanderen']

In [166]:
cols_to_drop = [
    'account_account_id',
    'account_adres_geografische_regio',
    'afspraak_afspraak_id',
    'afspraak_eindtijd',
    'activiteitvereistcontact_activityid_id',
    'activiteitvereistcontact_reqattendee',
    'account_oprichtingsdatum',
    'aangemaaktop',
    'gewijzigddoor',
    'gewijzigdop',
    'viewedon',
    'visitorkey',
    'webcontent',
    'account_activiteitscode_account',
    'account_activiteitscode_activiteitscode',
    'account_activiteitscode_inf_account_inf_activiteitscodeid',
    'activiteitscode_activiteitscode_id',
    'contact_contactpersoon_id',
    'functie_functie_id',
    'contactfunctie_contactpersoon',
    'persoon_persoon_id',
    'campagne_campagne_id',
    'campagne_einddatum',
    'campagne_startdatum',
    'sessie_eind_datum_tijd',
    'sessie_start_datum_tijd',
    'sessie_sessie_id',
    'inschrijving_datum_inschrijving',
    'inschrijving_inschrijving_id',
    'visit_visit_id',
    'visit_aangemaakt_op',
    'visit_gewijzigd_op',
    'pageview_id',
    'account_industriezone_naam_',
    'account_adres_provincie',
    'account_reden_van_status',
    'account_status',
    'account_voka_nr_',
    'account_adres_land',
    'activiteitscode_status',
    'status',
    'redenvanstatus',
    'type',
    'url',
    'operatingsystem',
    'financieledata_gewijzigd_op',
    'persoon_persoonsnr_',
    'persoon_reden_van_status',
    'persoon_web_login',
    'persoon_mail_regio_antwerpen_waasland',
    'persoon_mail_regio_west_vlaanderen',
    'persoon_mail_regio_vlaams_brabant',
    'persoon_mail_regio_mechelen_kempen',
    'persoon_mail_regio_limburg',
    'persoon_mail_regio_brussel_hoofdstedelijk_gewest',
    'persoon_mail_regio_antwerpen_waasland',
    'persoon_mail_regio_oost_vlaanderen',
    'persoon_mail_regio_voka_nationaal',
    'persoon_mail_thema_duurzaamheid',
    'persoon_mail_thema_financieel_fiscaal',
    'persoon_mail_thema_innovatie',
    'persoon_mail_thema_internationaal_ondernemen',
    'persoon_mail_thema_mobiliteit',
    'persoon_mail_thema_omgeving',
    'persoon_mail_thema_sales_marketing_communicatie',
    'persoon_mail_thema_strategie_en_algemeen_management',
    'persoon_mail_thema_talent',
    'persoon_mail_thema_welzijn',
    'persoon_mail_type_bevraging',
    'persoon_mail_type_communities_en_projecten',
    'persoon_mail_type_netwerkevenementen',
    'persoon_mail_type_nieuwsbrieven',
    'persoon_mail_type_opleidingen',
    'persoon_mail_type_persberichten_belangrijke_meldingen',
    'campagne_campagne_nr',
    'campagne_status',
    'campagne_naam_in_email',
    'campagne_url_voka_be',
    'sessie_sessie_nr_',
    'visit_adobe_reader',
    'visit_browser',
    'visit_contact_naam_',
    'visit_containssocialprofile',
    'visit_email_send',
    'visit_ended_on',
    'visit_entry_page',
    'visit_exit_page',
    'visit_ip_address',
    'visit_ip_organization',
    'visit_keywords',
    'visit_ip_longitude',
    'visit_ip_latitude',
    'visit_operating_system',
    'visit_started_on',
    'visit_time',
    'browser',
    'time',
    'visit_referrer',
    'visit_referrer_type',
    
]

In [167]:
data.drop(cols_to_drop, axis=1, inplace=True)
data.drop_duplicates(inplace=True)
data.shape

(2055938, 46)

### Adres kolommen samenvoegen waar mogelijk

In [ ]:
# voeg account_adres_geografische_subregio, account_adres_plaats samen tot account_adres
data['account_adres'] = data['account_adres_geografische_subregio'] + ', ' + data['account_adres_plaats']
# remove the (O.-Vl.) from the string
data['account_adres'] = data['account_adres'].apply(lambda x: re.sub(r'\(.*\)', '', x))
data.drop(['account_adres_geografische_subregio', 'account_adres_plaats'], axis=1, inplace=True)

### Type kolommen veranderen naar juiste dtypes

In [201]:
def change_dtype(col, dtype_first, dtype_to, options=None):
    try:
        if dtype_first == 'object' and dtype_to == 'float64':
            data[col] = data[col].astype('str')
            data[col] = data[col].apply(lambda x: x.replace('unknown', '-1'))
            data[col] = data[col].apply(lambda x: x.replace(',', '.'))
            data[col] = data[col].astype(dtype_to)
        
        if dtype_first == 'object' and dtype_to =='int64':
            data[col] = data[col].astype('str')
            data[col] = data[col].apply(lambda x: x.replace('unknown', '-1'))
            data[col] = data[col].astype(dtype_to)
        
        if dtype_first == 'object' and dtype_to =='onehot':
            data[col] = data[col].astype('str')
            data[col] = data[col].apply(lambda x: x.replace('unknown', '-1'))
            data.replace({col: {options[0]: 1, options[1]: 0}}, inplace=True)
    except:
        print("Error: kolommen zijn misschien al omgezet naar het gewenste datatype")


In [202]:
change_dtype('financieledata_fte', 'object', 'float64')
change_dtype('financieledata_toegevoegde_waarde', 'object', 'float64')
change_dtype('contact_status', 'object', 'onehot', ['Actief', 'Inactief'])

In [203]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2055938 entries, 0 to 3500991
Data columns (total 45 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   account_adres_postcode             int64  
 1   account_is_voka_entiteit           int64  
 2   account_ondernemingsaard           object 
 3   account_ondernemingstype           object 
 4   account_primaire_activiteit        object 
 5   activiteitscode_naam               object 
 6   financieledata_boekjaar            int64  
 7   financieledata_aantal_maanden      float64
 8   financieledata_toegevoegde_waarde  float64
 9   financieledata_fte                 float64
 10  contact_functietitel               object 
 11  contact_status                     int64  
 12  contact_voka_medewerker            int64  
 13  functie_naam                       object 
 14  persoon_marketingcommunicatie      object 
 15  campagne_naam                      object 
 16  campagne_reden_van_stat

In [207]:
data.iloc[10000]

account_adres_postcode                                                            9220
account_is_voka_entiteit                                                             0
account_ondernemingsaard                                          Productie & Diensten
account_ondernemingstype                                                Familiebedrijf
account_primaire_activiteit                                             Ijzer en staal
activiteitscode_naam                                                    Ijzer en staal
financieledata_boekjaar                                                           2011
financieledata_aantal_maanden                                                     12.0
financieledata_toegevoegde_waarde                                                 -1.0
financieledata_fte                                                                49.0
contact_functietitel                                                               CEO
contact_status                             